In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2021/train.csv
/kaggle/input/tabular-playground-series-apr-2021/test.csv


In [2]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


In [4]:
train.isnull()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   PassengerId  100000 non-null  int64  
 1   Survived     100000 non-null  int64  
 2   Pclass       100000 non-null  int64  
 3   Name         100000 non-null  object 
 4   Sex          100000 non-null  object 
 5   Age          96708 non-null   float64
 6   SibSp        100000 non-null  int64  
 7   Parch        100000 non-null  int64  
 8   Ticket       95377 non-null   object 
 9   Fare         99866 non-null   float64
 10  Cabin        32134 non-null   object 
 11  Embarked     99750 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 9.2+ MB


In [5]:
#Eliminando a coluna Cabin
del train['Cabin']

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,S


In [7]:
#Importante: RandomForest trabalha apenas com numeros.
from sklearn.ensemble import RandomForestClassifier
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

In [8]:
# Male = 0 ; Female = 1,
variables = ['Sex_bin', 'Age']

In [9]:
def change_sex(value):
    if value == 'female':
        return 1
    else:
        return 0
    
train['Sex_bin'] = train['Sex'].map(change_sex)
train.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Sex_bin
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,S,0
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,S,0
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,S,0
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,S,0
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,S,0
5,5,0,2,"Sassano, Jonathan",male,35.00,0,0,13363,6.71,S,0
6,6,0,3,"Conway, Jose",male,7.00,0,0,A/5,9.77,S,0
7,7,1,3,"Werner, Linda",female,6.00,1,2,434426,31.50,S,1
8,8,0,1,"Wardlaw, Michael",male,27.00,2,1,474849,73.02,S,0
9,9,0,2,"Greigo, Rudy",male,66.00,0,0,6981,9.14,C,0


In [10]:
variaveis = ['Sex_bin', 'Age']

In [11]:
X = train[variaveis]
y = train['Survived']

In [12]:
#Completando NaN com -1, pois Sex_bin e Age não possui -1
X = X.fillna(-1)
modelo.fit(X,y)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [13]:
test['Sex_bin'] = test['Sex'].map(change_sex)#transformando sex para sex_bin
X_prev = test[variaveis]
X_prev = X_prev.fillna(-1)
X_prev.head()

,Sex_bin,Age
0,0,19.0
1,1,53.0
2,1,19.0
3,0,25.0
4,1,17.0


In [14]:
p = modelo.predict(X_prev)
p

array([0, 1, 1, ..., 0, 1, 1])

In [15]:
#Criar estruturas de dados com valores = p
sub = pd.Series(p, index=test['PassengerId'], name='Survived')
sub.shape

(100000,)

In [16]:
sub.to_csv("First_model.csv", header=True)

In [17]:
!head -n10 First_model.csv

PassengerId,Survived
100000,0
100001,1
100002,1
100003,0
100004,1
100005,0
100006,1
100007,0
100008,1
